In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import graspologic as gl
import pickle
import pandas as pd

/home/gwyneth-local/anaconda3/envs/celegans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('data/pruned_graph.gpickle', 'rb') as f:
    G = pickle.load(f)

In [3]:
sensory_neurons = [node for node in G.nodes if G.nodes[node]['neuron type'] == 'Sens.']
interneurons = [node for node in G.nodes if G.nodes[node]['neuron type'] == 'Inter.']
motor_neurons = [node for node in G.nodes if G.nodes[node]['neuron type'] == 'Mot.']
print(len(sensory_neurons))
print(len(interneurons))
print(len(motor_neurons))

53
32
32


In [4]:
# pass 1: filter to only sensory neurons that have a direct struc connection to interneuron
filt1_sens = {}
filt1_int = set()
for sn in sensory_neurons:
    for itn in interneurons:
        if G.has_edge(sn, itn) and G[sn][itn]['has_stx']:
            if sn not in filt1_sens:
                filt1_sens[sn] = []
            filt1_sens[sn].append(itn)
            filt1_int.add(itn)

# pass 2: filter interneurons to only those that have a direct struc connection to motor neuron OR interneuron and filter sensory neurons to only those that
# have struc connection to a filter interneuron
filt2_int = set()
filt_motor_neurons = set()
for itn in filt1_int:
    for mn in motor_neurons+interneurons:
        if G.has_edge(itn, mn) and G[itn][mn]['has_stx']:
            filt_motor_neurons.add(mn)
            filt2_int.add(itn)
    if itn not in filt2_int:
        for sn in filt1_sens:
            if itn in filt1_sens[sn]:
                filt1_sens[sn].remove(itn)
filt_sens_neurons = {sn for sn in filt1_sens if len(filt1_sens[sn]) > 0}
filt_interneurons = filt2_int

print(len(filt_sens_neurons))
print(len(filt_interneurons))
print(len(filt_motor_neurons))

50
23
51


In [5]:
# checks

def check_motor_filt(motor_neurons, interneurons):
    for mn in motor_neurons:
        if not any(G.has_edge(itn, mn) for itn in interneurons):
            print(f"Motor neuron {mn} has no connection to any filtered interneurons.")
            return False
    return True

def check_inter_filt(interneurons, sensory_neurons):
    for itn in interneurons:
        if not any(G.has_edge(sn, itn) for sn in sensory_neurons):
            print(f"Interneuron {itn} has no connection to any filtered sensory neurons.")
            return False
        if not any(G.has_edge(itn, mn) for mn in motor_neurons):
            print(f"Interneuron {itn} has no connection to any filtered motor neurons.")
            return False
    return True

def check_sens_filt(sensory_neurons, interneurons):
    for sn in sensory_neurons:
        if not any(G.has_edge(sn, itn) for itn in interneurons):
            print(f"Sensor neuron {sn} has no connection to any filtered interneurons.")
            return False
    return True

print(check_motor_filt(filt_motor_neurons, filt_interneurons))
print(check_inter_filt(filt_interneurons, filt_sens_neurons))
print(check_sens_filt(filt_sens_neurons, filt_interneurons))
    

True
Interneuron RIFR has no connection to any filtered motor neurons.
False
True


In [6]:
# for each sensory neurons, get most similar sensory neuron in terms of functional output (metric: bits different)

def get_func_output(ingoing, possible_outgoing, G):
    """get functional output vector as a binary list of connections to a set of outgoing neurons"""
    func_output = []
    for outgoing in possible_outgoing:
        if G.has_edge(ingoing, outgoing) and G[ingoing][outgoing]['fx_weight'] > 0:
            func_output.append(1)
        else:
            func_output.append(0)
    return np.array(func_output)

def get_struc_output(ingoing, possible_outgoing, G):
    """get structural output vector as a binary list of connections to a set of outgoing neurons"""
    func_output = []
    for outgoing in possible_outgoing:
        if G.has_edge(ingoing, outgoing) and G[ingoing][outgoing]['stx_weight'] > 0:
            func_output.append(1)
        else:
            func_output.append(0)
    return np.array(func_output)

def output_distance(v1, v2):
    return np.sum(v1 != v2)

# sens_func_outputs = {sn: get_func_output(sn, interneurons, G) for sn in filt_sens_neurons}
sens_func_outputs = {sn: get_struc_output(sn, interneurons, G) for sn in filt_sens_neurons}
sens_to_min_dist_sens = {}
for sn in filt_sens_neurons:
    without_self_outputs = {key: value for key, value in sens_func_outputs.items() if key != sn}
    min_dist_sens = min(without_self_outputs, key=lambda x: output_distance(sens_func_outputs[sn], without_self_outputs[x]))
    min_dist = output_distance(sens_func_outputs[sn], without_self_outputs[min_dist_sens])
    sens_to_min_dist_sens[sn] = (min_dist_sens, min_dist)

# convert sens_to_min_dist_sens to a pandas df
closest_sens_df = pd.DataFrame(columns=['neuron_name', 'closest_neuron', 'dist'])
for sn, infos in sens_to_min_dist_sens.items():
    neuron_name, closest_neuron, dist = sn, infos[0], infos[1]
    closest_sens_df.loc[len(closest_sens_df)] = {'neuron_name': neuron_name, 'closest_neuron': closest_neuron, 'dist': dist} 

closest_sens_df

neuron_name closest_neuron  dist
0         ASEL           AWCL     3
1         AFDR           ASIR     1
2         AWAR           AWBR     2
3         AUAL           URXL     4
4         AFDL           AWAL     1
5         AUAR           BAGL     2
6         AWCR           AWCL     1
7         ASKR           ASGR     0
8        URYDL          URYVL     4
9         URBR           AFDR     3
10        ADFL           AWBL     3
11        ASIL           ASKR     1
12       IL2DR          OLQVR     0
13        URBL           AFDR     4
14       SAADR          SAADL     2
15        AWBL           URXL     2
16        URXL           AWBL     2
17       SAADL          SAAVL     1
18        ASHR           ADLR     8
19        ASGR           ASKR     0
20       SAAVL          SAADL     1
21        AWBR           AWAR     2
22        BAGR           AUAL     5
23       URYVL           OLLL     1
24        OLLL          URYVL     1
25       SAAVR          SAADR     2
26        ADLR           ASKR     5
27       IL2VL           ASKR     3
28        ASHL           ADLL     3
29        ADLL           ASHL     3
30        AWCL           AWCR     1
31        ASGL           ASIL     2
32       OLQVR          IL2DR     0
33         AQR           FLPR     7
34       IL2DL          IL2DR     3
35       URYDR           OLLR     2
36        ASER           AWCR     1
37        OLLR          URYDR     2
38        ASIR           AFDR     1
39        ASJR           AFDR     2
40        FLPR          URYVR     4
41        URXR           BAGL     4
42       URYVR           FLPR     4
43        IL2R          IL2DR     0
44        AWAL           AFDL     1
45       OLQDL          IL2DR     0
46        ASKL           ASIL     3
47        FLPL          URYDL     6
48        IL2L          IL2DR     0
49        BAGL           AUAR     2

In [35]:
# candidate neurons with identical 
s1 = 'BAGR'
s2 = 'URXL'
def get_feedforward_subgrapg(s1, s2):
    g = nx.DiGraph()
    for sn in [s1, s2]:
        g.add_node(sn, neuron_type='L1')
        for itn in filt_interneurons:
            if G.has_edge(sn, itn) and G[sn][itn]['has_stx']:
                g.add_node(itn, neuron_type='L2')
                g.add_edge(sn, itn, stx_weight = G[sn][itn]['stx_weight'])
    g_inter = [node for node in g.nodes if g.nodes[node]['neuron_type'] == 'L2']
    for mn in filt_motor_neurons:
        if mn in g.nodes:
            continue
        for itn in g_inter:
            if G.has_edge(itn, mn) and G[itn][mn]['has_stx']:
                g.add_node(mn, neuron_type='L3')
                g.add_edge(itn, mn, stx_weight = G[itn][mn]['stx_weight'])
    return g

g = get_feedforward_subgrapg(s1, s2)

In [36]:
pos = {}
layer_gap = 1.5 

for i, node in enumerate([n for n in g.nodes if g.nodes[n]['neuron_type'] == 'L1']):
    pos[node] = (i, 2 * layer_gap)

for i, node in enumerate([n for n in g.nodes if g.nodes[n]['neuron_type'] == 'L2']):
    pos[node] = (i, layer_gap)

third_layer = [n for n in g.nodes if g.nodes[n]['neuron_type'] == 'L3']
# sort third layer nodes by incoming degree
third_layer.sort(key=lambda x: g.in_degree(x), reverse=True)
for i, node in enumerate(third_layer[:6]):
    pos[node] = (i, 0)

g_s = nx.subgraph(g, [n for n in g.nodes if g.nodes[n]['neuron_type'] == 'L1' or g.nodes[n]['neuron_type'] == 'L2' or (n in third_layer[:6])])

# Get edge weights for coloring
edges = g_s.edges(data=True)
print(edges)
weights = [edge[2]['stx_weight'] for edge in edges]

# Normalize weights for color mapping
norm = plt.Normalize(vmin=min(weights), vmax=max(weights))

plt.figure(figsize=(8, 5)) 
ax = plt.gca()  # Get the current axes
nx.draw(
    g_s, pos, with_labels=True, node_size=500, font_size=6,
    node_color=[{'L1': 'lightblue', 'L2': 'lightgreen', 'L3': 'lightcoral'}[g.nodes[n]['neuron_type']] for n in g_s.nodes],
    edge_color=weights, edge_cmap=plt.cm.GnBu, edge_vmin=-8, edge_vmax=max(weights),
    ax=ax  # Pass the axes to the draw function
)

# Add color bar
sm = plt.cm.ScalarMappable(cmap=plt.cm.GnBu, norm=plt.Normalize(vmin=-8, vmax=max(weights)))
sm.set_array([])
plt.colorbar(sm, ax=ax, label='stx_weight')  # Associate the colorbar with the axes

plt.title(f"Feedforward Pathway from {s1} and {s2}")
plt.show()

[('RIAL', 'RIAR', {'stx_weight': 48.0}), ('RIAL', 'SMDVL', {'stx_weight': 17.0}), ('AIBR', 'AVER', {'stx_weight': 1.0}), ('AIBR', 'AVEL', {'stx_weight': 2.0}), ('AIBR', 'AVAR', {'stx_weight': 4.0}), ('AIBR', 'SMDVL', {'stx_weight': 6.0}), ('AIBR', 'AIZL', {'stx_weight': 2.0}), ('RIR', 'AVER', {'stx_weight': 1.0}), ('RIR', 'AVEL', {'stx_weight': 1.0}), ('RIR', 'RIAR', {'stx_weight': 9.0}), ('RIR', 'AIZL', {'stx_weight': 10.0}), ('AIYL', 'RIAR', {'stx_weight': 2.0}), ('AIYL', 'AIZL', {'stx_weight': 10.0}), ('URXL', 'RIAL', {'stx_weight': 21.0}), ('URXL', 'RIR', {'stx_weight': 7.0}), ('URXL', 'RIGL', {'stx_weight': 10.0}), ('RIBL', 'AVER', {'stx_weight': 10.0}), ('RIBL', 'AVEL', {'stx_weight': 6.0}), ('RIBL', 'AVAR', {'stx_weight': 2.0}), ('RIBL', 'AIZL', {'stx_weight': 1.0}), ('BAGR', 'AIYL', {'stx_weight': 2.0}), ('BAGR', 'RIAL', {'stx_weight': 19.0}), ('BAGR', 'RIBR', {'stx_weight': 2.0}), ('BAGR', 'AIBR', {'stx_weight': 2.0}), ('BAGR', 'RIBL', {'stx_weight': 21.0}), ('BAGR', 'RIR', {'

In [10]:
def calc_listening_breadth_from_subgraph(g: nx.DiGraph):
    third_layer = [n for n in g.nodes if g.nodes[n]['neuron_type'] == 'L3']
    max_node = max(third_layer, key = lambda x: g.in_degree(x))
    return g.in_degree(max_node)

def calc_listening_breadth(s1, s2):
    g = get_feedforward_subgrapg(s1, s2)
    return calc_listening_breadth_from_subgraph(g)

closest_sens_df['listening_breadth'] = closest_sens_df.apply(lambda row: calc_listening_breadth(row['neuron_name'], row['closest_neuron']), axis=1)
closest_sens_df.sort_values(by='listening_breadth', ascending=False)

neuron_name closest_neuron  dist  listening_breadth
1         BAGR           URXL     5                  5
30        ASHR           ADLR     8                  5
49        ASEL           ASER     3                  5
11        AWCR           ASER     1                  4
9          AQR          URYVR     7                  4
44        ASKL           ASIL     3                  4
24        AWCL           AWCR     1                  4
33        ADFL           AWBL     3                  4
46        URXR           BAGL     4                  4
0         ASER           AWCR     1                  4
18       SAADL          SAAVL     1                  3
20        ADLL           ASHL     3                  3
3         AUAR           BAGL     2                  3
2         URXL           AWBL     2                  3
12        AWBL           URXL     2                  3
14        FLPL          URYDL     6                  3
38        FLPR          URYVR     4                  3
45       SAAVL          SAADL     1                  3
37        AUAL           URXL     4                  3
19        ASHL           ADLL     3                  3
36       URYVR           FLPR     4                  3
42        BAGL           AUAR     2                  3
40        AFDR           ASIR     1                  2
47        AFDL           AWAL     1                  2
8         URBR           ASKR     3                  2
17        ASIR           ASKR     1                  2
4         ASIL           ASKR     1                  2
7         AWBR           AWAR     2                  2
41        AWAR           AWBR     2                  2
15       SAAVR          SAADL     2                  2
10       IL2DL           IL2L     3                  2
6         ADLR           ASKR     5                  2
21       IL2VL           ASKR     3                  2
22        ASGL           ASIL     2                  2
29       SAADR          SAAVR     2                  2
27        ASJR           ASKR     2                  2
23        AWAL           AFDL     1                  2
25        URBL           ASKR     4                  2
16        OLLR          URYDR     2                  1
13        ASGR           ASKR     0                  1
5         ASKR           ASGR     0                  1
28        IL2L           IL2R     0                  1
32       OLQVR           IL2L     0                  1
31        IL2R           IL2L     0                  1
26        OLLL          URYVL     1                  1
35       URYDL          URYVL     4                  1
39       URYVL           OLLL     1                  1
34       OLQDL           IL2L     0                  1
43       IL2DR           IL2L     0                  1
48       URYDR           OLLR     2                  1

In [116]:
motor_neuron_from_sens_paths = {}
for mn in g.nodes():
    if g.nodes[mn]['neuron_type'] == 'Mot.':
        connected_sens = []
        if nx.has_path(g, s1, mn):
            connected_sens.append(s1)
        if nx.has_path(g, s2, mn):
            connected_sens.append(s2)
        motor_neuron_from_sens_paths[mn] = connected_sens

print(motor_neuron_from_sens_paths)

{'RMDL': ['SAAVR', 'SAADR'], 'RMDR': ['SAAVR', 'SAADR'], 'SMDVR': ['SAAVR', 'SAADR'], 'SMDVL': ['SAAVR', 'SAADR'], 'SMDDL': ['SAAVR', 'SAADR'], 'SMDDR': ['SAAVR', 'SAADR'], 'RMFL': ['SAAVR', 'SAADR'], 'RMFR': ['SAAVR', 'SAADR']}


In [117]:
s1_activations = []
s2_activations = []

for mn in motor_neuron_from_sens_paths:
    if G.has_edge(s1, mn) and G[s1][mn]['has_fx']:
        s1_activations.append(mn)
    if G.has_edge(s2, mn) and G[s2][mn]['has_fx']:
        s2_activations.append(mn)

print("S1:", s1_activations)
print("S2:", s2_activations)
itn_activations = {}
for itn in g.nodes:
    if g.nodes[itn]['neuron_type'] == 'Inter.':
        if G.has_edge(s1, itn) and G[s1][itn]['has_fx']:
            s1_activations.append(itn)
        if G.has_edge(s2, itn) and G[s2][itn]['has_fx']:
            s2_activations.append(itn)
        if itn not in itn_activations:
            itn_activations[itn] = []
        for mn in motor_neuron_from_sens_paths:
            if G.has_edge(itn, mn) and G[itn][mn]['has_fx']:
                itn_activations[itn].append(mn)
itn_activations

S1: ['RMDR', 'SMDVL']
S2: ['SMDDL', 'SMDDR', 'RMFL']


{'RIML': ['SMDDL', 'RMFR'], 'RIMR': ['RMDR', 'RMFR']}

In [118]:
# put everything into pandas df

motor_s1s2_df = pd.DataFrame(columns=['neuron_name', 'path_from_s1', 'path_from_s2', 's1_fx', 's2_fx'])
for mn in motor_neuron_from_sens_paths:
    path_from_s1 = s1 in motor_neuron_from_sens_paths[mn]
    path_from_s2 = s2 in motor_neuron_from_sens_paths[mn]
    s1_fx = mn in s1_activations
    s2_fx = mn in s2_activations
    motor_s1s2_df.loc[len(motor_s1s2_df)] = {'neuron_name': mn, 'path_from_s1': path_from_s1, 'path_from_s2': path_from_s2, 's1_fx': s1_fx, 's2_fx': s2_fx}
    
motor_s1s2_df

neuron_name  path_from_s1  path_from_s2  s1_fx  s2_fx
0        RMDL          True          True  False  False
1        RMDR          True          True   True  False
2       SMDVR          True          True  False  False
3       SMDVL          True          True   True  False
4       SMDDL          True          True  False   True
5       SMDDR          True          True  False   True
6        RMFL          True          True  False   True
7        RMFR          True          True  False  False

In [119]:
# compute 2-connection column
motor_s1s2_df['2-path'] = (motor_s1s2_df['path_from_s1']) & (motor_s1s2_df['path_from_s2'])
motor_s1s2_df['2-fx'] = (motor_s1s2_df['s1_fx']) & (motor_s1s2_df['s2_fx'])
motor_s1s2_df

neuron_name  path_from_s1  path_from_s2  s1_fx  s2_fx  2-path   2-fx
0        RMDL          True          True  False  False    True  False
1        RMDR          True          True   True  False    True  False
2       SMDVR          True          True  False  False    True  False
3       SMDVL          True          True   True  False    True  False
4       SMDDL          True          True  False   True    True  False
5       SMDDR          True          True  False   True    True  False
6        RMFL          True          True  False   True    True  False
7        RMFR          True          True  False  False    True  False

In [120]:
correlation_matrix = motor_s1s2_df.drop(columns=['neuron_name']).corr(method='spearman')
print(correlation_matrix)

              path_from_s1  path_from_s2     s1_fx     s2_fx  2-path  2-fx
path_from_s1           NaN           NaN       NaN       NaN     NaN   NaN
path_from_s2           NaN           NaN       NaN       NaN     NaN   NaN
s1_fx                  NaN           NaN  1.000000 -0.447214     NaN   NaN
s2_fx                  NaN           NaN -0.447214  1.000000     NaN   NaN
2-path                 NaN           NaN       NaN       NaN     NaN   NaN
2-fx                   NaN           NaN       NaN       NaN     NaN   NaN


In [129]:
# for each pair of sensory neurons, get a vector of interneuron and gates

def get_interneuron_and_vec(s1, s2):
    base = [0] * len(filt_interneurons)
    for ix, itn in enumerate(filt_interneurons):
        if not G.has_edge(s1, itn) or not G.has_edge(s2, itn):
            continue
        if not G[s1][itn]['has_stx'] or not G[s2][itn]['has_stx']:
            continue
        base[ix] = True
    return np.array(base)

and_itn_vecs = np.ndarray(shape=(len(filt_sens_neurons), len(filt_sens_neurons), len(filt_interneurons)), dtype=bool)
for i, s1 in enumerate(filt_sens_neurons):
    for j, s2 in enumerate(filt_sens_neurons):
        if i == j:
            continue
        and_itn_vecs[i][j] = get_interneuron_and_vec(s1, s2)
        and_itn_vecs[j][i] = and_itn_vecs[i][j]

In [133]:
# project and gate vectors down onto motor neurons
def get_itn_to_mot_projection(itn_vec):
    mot_vec = np.zeros(len(filt_motor_neurons), dtype=bool)
    for ix, itn in enumerate(filt_interneurons):
        if itn_vec[ix]: # if active in itn_vec
            for ixm, mn in enumerate(filt_motor_neurons):
                if G.has_edge(itn, mn) and G[itn][mn]['has_stx']:
                    mot_vec[ixm] = True
    return mot_vec

motor_proj = np.ndarray(shape=(len(filt_sens_neurons), len(filt_sens_neurons), len(filt_motor_neurons)), dtype=bool)
for i, s1 in enumerate(filt_sens_neurons):
    for j, s2 in enumerate(filt_sens_neurons):
        if i == j:
            continue
        motor_proj[i][j] = get_itn_to_mot_projection(and_itn_vecs[i][j])
        motor_proj[j][i] = motor_proj[i][j]

In [145]:
# build data frame consisting of motor neuron activations for each pair of sensory neurons

motor_activation_df = pd.DataFrame(columns=['sensory_neuron_1', 'sensory_neuron_2', 'motor_activation', 'motor_projection'])

for i, s1 in enumerate(filt_sens_neurons):
    for j, s2 in enumerate(filt_sens_neurons):
        if i == j:
            continue
        # skip if opposite order already in motor_activation_df
        if not motor_activation_df[
            (motor_activation_df['sensory_neuron_1'] == s2) & 
            (motor_activation_df['sensory_neuron_2'] == s1)
        ].empty:
            continue
        mn_act_vec = np.zeros(len(filt_motor_neurons), dtype=bool)
        for ixm, mn in enumerate(filt_motor_neurons):
            # sens 1 active
            if G.has_edge(s1, mn) and G[s1][mn]['has_fx']:
                sens1_active = True
            else:
                sens1_active = False
            # sens 2 active
            if G.has_edge(s2, mn) and G[s2][mn]['has_fx']:
                sens2_active = True
            else:
                sens2_active = False
            mn_act_vec[ixm] = not (sens1_active or sens2_active)
            # add to df
        motor_activation_df.loc[len(motor_activation_df)] = {'sensory_neuron_1': s1, 'sensory_neuron_2': s2, 'motor_activation': mn_act_vec, 'motor_projection': motor_proj[i][j]}
            
motor_activation_df

sensory_neuron_1 sensory_neuron_2  \
0                ASHL             BAGR   
1                ASHL             URXR   
2                ASHL             ASGL   
3                ASHL             FLPR   
4                ASHL            IL2DR   
...               ...              ...   
1123             ASIR             URBR   
1124             ASIR             URXL   
1125             BAGL             URBR   
1126             BAGL             URXL   
1127             URBR             URXL   

                                       motor_activation  \
0     [True, True, True, True, True, True, True, Fal...   
1     [True, True, True, True, True, True, True, Fal...   
2     [True, True, True, True, True, True, True, Fal...   
3     [True, True, True, True, True, True, True, Fal...   
4     [True, True, True, True, True, True, True, Fal...   
...                                                 ...   
1123  [True, True, True, True, False, True, True, Tr...   
1124  [True, True, True, True, False, True, True, Fa...   
1125  [True, True, True, True, True, False, True, Tr...   
1126  [True, True, True, True, True, False, True, Fa...   
1127  [True, True, True, True, True, True, True, Fal...   

                                       motor_projection  
0     [True, True, False, True, False, True, False, ...  
1     [False, False, False, False, False, False, Fal...  
2     [False, True, True, True, True, False, False, ...  
3     [False, False, False, False, False, False, Fal...  
4     [False, False, False, False, False, False, Fal...  
...                                                 ...  
1123  [False, False, False, False, False, False, Fal...  
1124  [False, False, False, False, False, False, Fal...  
1125  [False, False, False, False, False, False, Fal...  
1126  [False, False, False, False, False, False, Fal...  
1127  [False, False, False, False, False, False, Fal...  

[1128 rows x 4 columns]

In [153]:
import numpy as np

A = np.stack(motor_activation_df["motor_activation"].values)
P = np.stack(motor_activation_df["motor_projection"].values)

r_global = np.corrcoef(A.ravel().astype(int),
                       P.ravel().astype(int))[0, 1]
print("overall Pearson r:", r_global)

overall Pearson r: -0.04545592320491755
